In [1]:
from datasets import load_dataset

raw_datasets = load_dataset("conll2003")

c:\Users\priks\anaconda3\envs\t\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [3]:
raw_datasets["train"][0]["tokens"],raw_datasets["train"][0]["ner_tags"]

(['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'],
 [3, 0, 7, 0, 0, 0, 7, 0, 0])

In [4]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [5]:
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [6]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

EU    rejects German call to boycott British lamb . 
B-ORG O       B-MISC O    O  O       B-MISC  O    O 


In [7]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [8]:
tokenizer.is_fast

True

In [9]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [10]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'EU',
 'rejects',
 'German',
 'call',
 'to',
 'boycott',
 'British',
 'la',
 '##mb',
 '.',
 '[SEP]']

In [11]:
inputs.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

In [12]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [13]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[3, 0, 7, 0, 0, 0, 7, 0, 0]
[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]


In [14]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [15]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

In [16]:
tokenized_datasets["train"][0]

{'input_ids': [101,
  7270,
  22961,
  1528,
  1840,
  1106,
  21423,
  1418,
  2495,
  12913,
  119,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]}

In [17]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [18]:
import evaluate

metric = evaluate.load("seqeval")

In [19]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [20]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=8
)

In [21]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [23]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [24]:
from transformers import get_scheduler

num_train_epochs = 20
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [25]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

In [26]:
import json
import os
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import torch
import time

# Define directories
output_dir = "bert-finetuned-ner-accelerate-checkpoint"
final_output_dir = "bert-finetuned-ner-accelerate"
metrics_output_file = os.path.join(output_dir, "metrics.json")
log_history_file = os.path.join(output_dir, "log_history.json")  # New file for log history

progress_bar = tqdm(range(num_training_steps))
all_metrics = {}  # To store metrics for each epoch
log_history = []  # To store state log history

# Define a helper function to handle non-serializable types
def convert_to_serializable(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        raise TypeError(f"Object of type {type(obj).__name__} is not JSON serializable")

for epoch in range(num_train_epochs):
    # Start tracking time
    epoch_start_time = time.time()

    # Training
    model.train()
    num_samples = 0
    num_steps = 0
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        num_samples += batch["input_ids"].size(0)
        num_steps += 1
        progress_bar.update(1)

    # End training time tracking
    epoch_training_time = time.time() - epoch_start_time

    # Evaluation
    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    # Compute metrics for the current epoch
    results = metric.compute()

    # Extract overall metrics
    overall_metrics = {
        key: results[f"overall_{key}"]
        for key in ["precision", "recall", "f1", "accuracy"]
    }

    # Extract per-label metrics
    per_label_metrics = {
        label: {
            "precision": metrics["precision"],
            "recall": metrics["recall"],
            "f1": metrics["f1"],
            "number": metrics["number"],
        }
        for label, metrics in results.items() if not label.startswith("overall")
    }

    # Compute runtime metrics
    samples_per_second = num_samples / epoch_training_time
    steps_per_second = num_steps / epoch_training_time

    # Combine all metrics for this epoch
    epoch_metrics = {
        "overall": overall_metrics,
        "per_label": per_label_metrics,
        "runtime": {
            "epoch": epoch,
            "training_time": epoch_training_time,
            "samples_per_second": samples_per_second,
            "steps_per_second": steps_per_second,
        },
    }
    all_metrics[f"epoch_{epoch}"] = epoch_metrics

    # Save metrics to the current epoch's folder
    epoch_output_dir = os.path.join(output_dir, f"epoch-{epoch}")
    accelerator.wait_for_everyone()
    if not os.path.exists(epoch_output_dir):
        os.makedirs(epoch_output_dir)
    epoch_metrics_file = os.path.join(epoch_output_dir, "metrics.json")
    with open(epoch_metrics_file, "w") as f:
        json.dump(epoch_metrics, f, indent=4, default=convert_to_serializable)

    # Update the global metrics file
    with open(metrics_output_file, "w") as f:
        json.dump(all_metrics, f, indent=4, default=convert_to_serializable)

    # Log state history (simulating `state.log_history`)
    log_entry = {
        "epoch": epoch,
        "precision": overall_metrics["precision"],
        "recall": overall_metrics["recall"],
        "f1": overall_metrics["f1"],
        "accuracy": overall_metrics["accuracy"],
        "training_time": epoch_training_time,
    }
    log_history.append(log_entry)

    # Save the log history to a file after each epoch
    with open(log_history_file, "w") as f:
        json.dump(log_history, f, indent=4, default=convert_to_serializable)

    # Clean notebook-friendly output
    print(f"Epoch {epoch}: Precision={overall_metrics['precision']:.4f}, "
          f"Recall={overall_metrics['recall']:.4f}, F1={overall_metrics['f1']:.4f}, "
          f"Accuracy={overall_metrics['accuracy']:.4f}, "
          f"Samples/sec={samples_per_second:.2f}, Steps/sec={steps_per_second:.2f}")

    # Display per-label metrics as a table
    per_label_df = pd.DataFrame(per_label_metrics).T
    per_label_df.index.name = "Label"
    per_label_df.reset_index(inplace=True)
    display(per_label_df)

    # Save the model checkpoint for the current epoch
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(epoch_output_dir, save_function=accelerator.save)

# Save the final model, tokenizer, metrics, and log history
if accelerator.is_main_process:
    if not os.path.exists(final_output_dir):
        os.makedirs(final_output_dir)

    print("Saving the final model...")

    # Save the model and tokenizer
    unwrapped_model.save_pretrained(final_output_dir, save_function=accelerator.save)
    tokenizer.save_pretrained(final_output_dir)

    # Save final metrics to the final output directory
    final_metrics_file = os.path.join(final_output_dir, "metrics.json")
    with open(final_metrics_file, "w") as f:
        json.dump(all_metrics, f, indent=4, default=convert_to_serializable)

    # Save final log history to the final output directory
    final_log_history_file = os.path.join(final_output_dir, "log_history.json")
    with open(final_log_history_file, "w") as f:
        json.dump(log_history, f, indent=4, default=convert_to_serializable)

    print(f"Final model, metrics, and log history saved to {final_output_dir}")



  0%|          | 0/35120 [00:00<?, ?it/s]c:\Users\priks\anaconda3\envs\t\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
  5%|▌         | 1756/35120 [02:41<52:02, 10.69it/s]  

Epoch 0: Precision=0.9261, Recall=0.8996, F1=0.9127, Accuracy=0.9823, Samples/sec=87.20, Steps/sec=10.90


,Label,precision,recall,f1,number
0,LOC,0.939031,0.942623,0.940824,1830.0
1,MISC,0.840564,0.786802,0.812795,985.0
2,ORG,0.923192,0.863922,0.892574,1433.0
3,PER,0.958198,0.944355,0.951226,1869.0


 10%|▉         | 3511/35120 [05:29<49:03, 10.74it/s]   

Epoch 1: Precision=0.9401, Recall=0.9113, F1=0.9254, Accuracy=0.9845, Samples/sec=87.33, Steps/sec=10.92


...

,Label,precision,recall,f1,number
0,LOC,0.965161,0.966739,0.965949,1834.0
1,MISC,0.907809,0.894231,0.900969,936.0
2,ORG,0.928412,0.904139,0.916115,1377.0
3,PER,0.979370,0.963161,0.971198,1873.0


100%|█████████▉| 35119/35120 [56:17<00:00, 11.05it/s]

Epoch 19: Precision=0.9517, Recall=0.9387, F1=0.9452, Accuracy=0.9862, Samples/sec=88.09, Steps/sec=11.02


,Label,precision,recall,f1,number
0,LOC,0.964616,0.965142,0.964879,1836.0
1,MISC,0.907809,0.891374,0.899516,939.0
2,ORG,0.926174,0.902616,0.914244,1376.0
3,PER,0.979370,0.963161,0.971198,1873.0


Saving the final model...
Final model, metrics, and log history saved to bert-finetuned-ner-accelerate


In [27]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "bert-finetuned-ner-accelerate"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'entity_group': 'PER',
  'score': 0.9999658,
  'word': 'Sylvain',
  'start': 11,
  'end': 18},
 {'entity_group': 'ORG',
  'score': 0.99996203,
  'word': 'Hugging Face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': 0.9999542,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

In [28]:
import pandas as pd
from transformers import AutoModelForTokenClassification, AutoTokenizer
from tqdm.auto import tqdm
from accelerate import Accelerator
import torch
import json
import numpy as np

accelerator = Accelerator()

# Load the trained model and tokenizer
final_output_dir = "bert-finetuned-ner-accelerate"
model = AutoModelForTokenClassification.from_pretrained(final_output_dir)
tokenizer = AutoTokenizer.from_pretrained(final_output_dir)

# Prepare the test dataloader
test_dataloader = DataLoader(
    tokenized_datasets["test"], collate_fn=data_collator, batch_size=8
)

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define a helper function to handle non-serializable types
def convert_to_serializable(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        raise TypeError(f"Object of type {type(obj).__name__} is not JSON serializable")

# Testing loop
model.eval()
progress_bar = tqdm(test_dataloader)
metric_results = {}

for batch in progress_bar:
    with torch.no_grad():
        # Move batch to device
        batch = {key: val.to(device) for key, val in batch.items()}
        
        # Forward pass
        outputs = model(**batch)
        
        # Get predictions and labels
        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]
        
        # Pad predictions and labels for distributed gathering (if necessary)
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        # Gather predictions and labels across all processes
        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)
        
        # Postprocess to remove padding and decode tokens
        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
        
        # Add predictions and references to the metric
        metric.add_batch(predictions=true_predictions, references=true_labels)

# Compute final results
results = metric.compute()

# Extract overall metrics
overall_metrics = {
    key: results[f"overall_{key}"]
    for key in ["precision", "recall", "f1", "accuracy"]
}
metric_results["overall"] = overall_metrics

# Extract per-label metrics
label_metrics = {
    label: {
        "precision": metrics["precision"],
        "recall": metrics["recall"],
        "f1": metrics["f1"],
        "number": metrics["number"]
    }
    for label, metrics in results.items() if not label.startswith("overall")
}
metric_results["per_label"] = label_metrics

# Display overall metrics
print("Overall Metrics:")
overall_df = pd.DataFrame([overall_metrics])
display(overall_df)

# Display per-label metrics
print("\nPer-Label Metrics:")
per_label_df = pd.DataFrame(label_metrics).T
per_label_df.index.name = "Label"
per_label_df.reset_index(inplace=True)
display(per_label_df)

# Save metrics to a JSON file
test_metrics_file = f"{final_output_dir}/test_metrics.json"
if accelerator.is_main_process:
    with open(test_metrics_file, "w") as f:
        json.dump(metric_results, f, indent=4, default=convert_to_serializable)
    print(f"Test results saved to {test_metrics_file}")


100%|██████████| 432/432 [00:08<00:00, 53.50it/s]


Overall Metrics:


,precision,recall,f1,accuracy
0,0.918201,0.896766,0.907357,0.972345



Per-Label Metrics:


,Label,precision,recall,f1,number
0,LOC,0.929856,0.921021,0.925418,1684.0
1,MISC,0.820513,0.750000,0.783673,768.0
2,ORG,0.906683,0.882767,0.894565,1706.0
3,PER,0.960421,0.955692,0.958051,1625.0


Test results saved to bert-finetuned-ner-accelerate/test_metrics.json


In [1]:
from transformers import pipeline

# Replace with your specific model checkpoint
model_checkpoint = "bert-finetuned-ner-accelerate"
token_classifier = pipeline(
    "token-classification", 
    model=model_checkpoint, 
    aggregation_strategy="simple"
)

# Example usage
result = token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")
print(result)

c:\Users\priks\anaconda3\envs\t\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'entity_group': 'PER', 'score': 0.9999658, 'word': 'Sylvain', 'start': 11, 'end': 18}, {'entity_group': 'ORG', 'score': 0.99996203, 'word': 'Hugging Face', 'start': 33, 'end': 45}, {'entity_group': 'LOC', 'score': 0.9999542, 'word': 'Brooklyn', 'start': 49, 'end': 57}]
